In [1]:
from mediawikiapi import MediaWikiAPI
from wikidata.client import Client
import json
import penman
from tqdm import tqdm
import pandas as pd
from itertools import chain
from collections import Counter
import concurrent.futures
from collections import defaultdict
import urllib.parse
import re
import time

start_time = time.time()
mediawikiapi = MediaWikiAPI()
client = Client()
end_time = time.time()
print(f"Initialization time: {end_time-start_time} seconds")

Initialization time: 0.27617669105529785 seconds


In [2]:
def find_wiki_names(graph,amr_string):
    wiki_names = []
    wiki_idx= []
    nodes_list = []
    roles = []
    rel_nodes = {}
    Salutations_list =[ "mr.", "mrs.", "ms.", "miss" , "sir" , "madam" , "mr", "mrs", "ms"]
    for i, triple in enumerate(graph.triples):

        if triple[1] == ':wiki':
            wiki_idx.append(i)
            node = triple[0]
            name_parts = []
            current_mod = []
            pointer = 1
            state_1 = False
            for idx , triple in enumerate(graph.triples):
                if triple[0] == node and triple[1] == ':instance':
                    # replace any '-' or '_' characters with a space
                    nodes_list.append(triple[2])
                    instance_node = triple[2]

                    top_node = triple[0]
                    if ":mod" in amr_string: # consider if :location as well
                        for triple in graph.triples:
                            if triple[1] == ':mod' and triple[0] == top_node:
                                rel_nodes[triple[2]] = None
                                state_1 = True  
                    if instance_node == 'person' :
                        if 'have-org-role-91' in amr_string:
                            # instance_node=''
                            pass
                        elif 'government-organization' in amr_string:
                            # if government-organization is the instance node, then we should not include the name of the government organization
                            # As it is the most well known name and government-organization will increase its perplexity
                            # instance_node=''
                            pass
                        elif 'country' in amr_string:
                            # if government-organization is the instance node, then we should not include the name of the government organization
                            # As it is the most well known name and government-organization will increase its perplexity
                            # instance_node=''
                            pass
                        else:
                            instance_node = '#del#'
                            continue

                state = False
                name_nodes = {}
                name= []
                # rel_nodes = []
                # rel_nodes = {}
                switch = False
                
                if triple[0] == node and triple[1] == ':name':
                    name_node = triple[2]
                    for k ,name_triple in enumerate(graph.triples[idx:]):
                        if name_triple[0] == name_node and name_triple[1].startswith(':op'):
                            name_parts.append(name_triple[2].strip('"'))
                            if state_1:
                                state = True
                            if idx+ k+2 < len(graph.triples):
                                if graph.triples[idx+k+2][2] == "have-org-role-91" and graph.triples[idx+k+1][2] == top_node:
                                    state = True
                                    switch = False
                                    # name_node_2 = name_triple[0]
                                    # rel_nodes.append(graph.triples[k+2][0])
                                    rel_nodes = {graph.triples[idx+ k+2][0]: None}
                                    roles.append(graph.triples[idx+k+2][0])
                                    continue
                                if graph.triples[idx + k+1][0] in roles and graph.triples[idx + k+1][1].startswith(':ARG') and graph.triples[idx + k+1][2] != graph.triples[idx + k+2][0]:
                                    state = True
                                    switch = False
                                    rel_nodes = {graph.triples[idx+ k+1][0]: None}
                                    continue
                                continue

                        if switch:
                            break
                        if state and name_triple[1].startswith(':ARG'):
                            if name_triple[0] in rel_nodes.keys() and name_triple[2] == graph.triples[k+1][0]:
                                if name_triple[2] in rel_nodes:
                                    continue
                                # rel_nodes.append(name_triple[2])
                                rel_nodes[name_triple[2]] = None
                            if top_node in rel_nodes.keys():
                                rel_nodes.pop(top_node)
                        # new attempt
                        if state:
                            for j ,name_triple_j in enumerate(graph.triples): 
                                # switch = True
                                if name_triple_j[1] == ':instance' and name_triple_j[0] in rel_nodes.keys():
                                    rel_nodes[name_triple_j[0]] = re.sub(r'-\d+$', '', name_triple_j[2])
                                    
                                    continue

                                if name_triple_j[1].startswith(':ARG'):
                                    if j+1 < len(graph.triples):
                                        if name_triple_j[2] != graph.triples[j+1][0]:
                                            continue

                                    if name_triple_j[0] in rel_nodes.keys():
                                        if name_triple_j[2] in rel_nodes.keys():
                                            continue
                                        # rel_nodes.append(name_triple_j[2])
                                        rel_nodes[name_triple_j[2]] = None
                                    if top_node in rel_nodes.keys():
                                        rel_nodes.pop(top_node)
                                        
                                if name_triple_j[0] in rel_nodes.keys() and name_triple_j[1] == ':name':
                                    if name_triple_j[2] in name_nodes.keys():
                                        continue
                                    name_nodes[name_triple_j[2]] = name_triple_j[0]

                                if name_triple_j[0] in name_nodes.keys() and name_triple_j[1].startswith(':op') :
                                    # if name_triple_j[2].strip('"') in name_parts:
                                    #     # print('test')
                                    #     continue
                                    # name_parts.append(name_triple_j[2].strip('"'))
                                    if j+1 != len(graph.triples):
                                        name.append(name_triple_j[2].strip('"'))
                                        if graph.triples[j+1][1].startswith(':op'):
                                            pass
                                        else:
                                            rel_nodes[name_nodes[name_triple_j[0]]] = ' '.join(name)
                                        # if graph.triples[j+1][1].startswith(':ARG'):
                                        #     switch = True
                                        #     break
                                    # switch = True
                                        name = []
                                    continue
                                if j+1 != len(graph.triples):
                                    continue
                                else:
                                    switch = True
                                    break
            for key , value in rel_nodes.items():
                if key in roles:
                    continue
                if value == None:
                    continue
                name_parts.append(value)
            for name_part in name_parts:
                if name_part.lower() in Salutations_list:
                    name_parts.remove(name_part)

            rel_nodes = {}
            # reverse the current_mod list
            current_mod = current_mod[::-1]
            full_name = instance_node + ' ' + ' '.join(current_mod +name_parts)
            wiki_names.append(full_name)
    return wiki_names , wiki_idx , nodes_list


def add_wiki_tag(graph, wiki_idx, wiki_names, nodes_list, entities_dict, mediawikiapi, client):
      """
      get the parent-child relationships
      """
      # Build parent-child relationship
      
      for i, content in enumerate(wiki_names):
            switch = True
            if '#del#' in content:
                continue
            try:
                if nodes_list[i] not in entities_dict.keys():
                    result = mediawikiapi.search(str(content), results=1)
                    # result = result.replace(' ', '_')
                    # print(result)
                    if result == []:
                        continue
                    try:
                        if result == mediawikiapi.page(str(list([result]))).title:
                            result = str(list([result]))
                        elif result == mediawikiapi.page(str([result])).title:
                            result = str([result])
                    except:
                        if result == mediawikiapi.page(str([result])).title:
                            result = str([result])
                        elif result == mediawikiapi.page(str(list([result]))).title:
                            result = str(list([result]))
                    copy = list(graph.triples[wiki_idx[i]])
                    try:
                        copy[2] = '"' + urllib.parse.unquote(mediawikiapi.page(result).url.split('/wiki/')[1]) + '"'
                    except:
                        copy[2] = '"' + urllib.parse.unquote(mediawikiapi.page(result).url.split('/wiki/')[1]) + '"'     
                    # copy[2] = '"' + urllib.parse.unquote(mediawikiapi.page(str(list([result]))).url.split('/wiki/')[1]) + '"'
                    graph.triples[wiki_idx[i]] = tuple(copy)
                    continue
                query = ' '.join(content.split()[1:])
                results0 = mediawikiapi.search(str(query), results=5)
                # print(results0)

                if results0 == []:
                    query = content.replace('-', ' ').replace('_', ' ')
                    switch = False
                    results0 = mediawikiapi.search(str(query), results=3)
                    if results0 == []:
                    #     if nodes_list[i] != 'person':
                    #         results0.append(nodes_list[i])
                        continue

                for result in results0:
                    # result = result.replace(' ', '_')
                    try:
                        if result == mediawikiapi.page(str(list([result]))).title:
                            result = str(list([result]))
                        elif result == mediawikiapi.page(str([result])).title:
                            result = str([result])
                        else:
                            continue
                    except:
                        if result == mediawikiapi.page(str([result])).title:
                            result = str([result])
                        elif result == mediawikiapi.page(str(list([result]))).title:
                            result = str(list([result]))
                        else:
                            continue
                    entities, count = get_wikidata_tags(result, mediawikiapi, client)
                    # if any entities in the entities_dict[nodes_list[i]] then pass
                    if any(entity in entities_dict[nodes_list[i]] for entity in entities):
                        copy = list(graph.triples[wiki_idx[i]]) 
                        try:
                            copy[2] = '"' + urllib.parse.unquote(mediawikiapi.page(result).url.split('/wiki/')[1]) + '"'
                        except:
                            copy[2] = '"' + urllib.parse.unquote(mediawikiapi.page(result).url.split('/wiki/')[1]) + '"'         
                        # copy[2] = '"' + urllib.parse.unquote(mediawikiapi.page(str(list([result]))).url.split('/wiki/')[1]) + '"'
                        graph.triples[wiki_idx[i]] = tuple(copy)
                        switch = False
                        break
                if switch:
                    query = content.replace('-', ' ').replace('_', ' ')
                    results0 = mediawikiapi.search(str(query), results=3)
                    if results0 == []:
                        print('Error wiki: ', content)
                        continue
                    for result in results0:
                        # result = result.replace(' ', '_')
                        try:
                            if result == mediawikiapi.page(str(list([result]))).title:
                                result = str(list([result]))
                            elif result == mediawikiapi.page(str([result])).title:
                                result = str([result])
                            else:
                                continue
                        except:
                            if result == mediawikiapi.page(str([result])).title:
                                result = str([result])
                            elif result == mediawikiapi.page(str(list([result]))).title:
                                result = str(list([result]))
                            else:
                                continue
                        entities, count = get_wikidata_tags(result, mediawikiapi, client)
                        if any(entity in entities_dict[nodes_list[i]] for entity in entities):

                            copy = list(graph.triples[wiki_idx[i]])
                            try:
                                copy[2] = '"' + urllib.parse.unquote(mediawikiapi.page(result).url.split('/wiki/')[1]) + '"'
                            except:
                                copy[2] = '"' + urllib.parse.unquote(mediawikiapi.page(result).url.split('/wiki/')[1]) + '"'     
                            # copy[2] = '"' + urllib.parse.unquote(mediawikiapi.page(str(list([result]))).url.split('/wiki/')[1]) + '"'
                            graph.triples[wiki_idx[i]] = tuple(copy)
                            break
                    continue
            except:
                print('Error wiki: ', content)
                continue
      return graph

def get_wikidata_tags(query, mediawikiapi, client,count=0):
    statement_list = ["P31", "P279","P10241","P361","P16","P3450", "P5138","P289","P452","P571", "P366" , "P1269"]
    entities= []


    # wiki_data = mediawikiapi.search(str(query), results=1)
    # wikidata_id = mediawikiapi.page(str(query)).pageprops["wikibase_item"]
    # try:
        
    #     wikidata_id = mediawikiapi.page(str(list([query]))).pageprops["wikibase_item"]
    # except:
    #     try:
    #         wikidata_id = mediawikiapi.page(str([query])).pageprops["wikibase_item"]
    #     except:
    #         print("Error wikidata: ", query)

    #         count += 1
    #         # the true value is to indicate that the no wikidata link was found 
    #         # as this is a very rare case, we can assume that the wiki tag is correct
    #         entities.append('true')
    #         return entities , count
    try:
        
        wikidata_id = mediawikiapi.page(query).pageprops["wikibase_item"]

    except:
        print("Error wikidata: ", query)
        # the true value is to indicate that the no wikidata link was found 
        # as this is a very rare case, we can assume that the wiki tag is correct
        entities.append('true')
        return entities , count
    entity = client.get(wikidata_id, load=True)
    # check if any entity has the statement_list keys
    target = None
    for statement in statement_list:
        if statement in entity.data['claims'].keys():
            target = statement
            break
    if target == None:
        return entities , count
    if target == "P31":
        for value in entity.data['claims'][target]:
            entities.append(value["mainsnak"]["datavalue"]["value"]["id"])
    else:
        try:
            subclass_entity = client.get(entity.data['claims'][target][0]["mainsnak"]["datavalue"]["value"]["id"], load=True)
            entities.append(subclass_entity.data['claims']["P31"][0]["mainsnak"]["datavalue"]["value"]["id"])
        except:
            try:
                entities.append(entity.data['claims'][target][0]["mainsnak"]["datavalue"]["value"]["id"])
            except:
                return [] , count
    return entities , count

def get_wiki(amr, dict_entities, mediawikiapi, client):
    if ':wiki' in amr:
        graph = penman.decode(amr)
        wiki_names , wiki_idx , nodes_list = find_wiki_names(graph,amr)
        graph = add_wiki_tag(graph, wiki_idx, wiki_names, nodes_list, dict_entities, mediawikiapi, client)
        amr = penman.encode(graph)
        # data[i] = amr_string
    return amr 
# Function to process each AMR
def get_wiki_with_index(index, amr, dict_entities, mediawikiapi, client):
    result = get_wiki(amr, dict_entities, mediawikiapi, client)
    return index, result

In [3]:
with open('/mnt/e/workspace/UCL_master_project/Get_wiki/AMR_without_wiki/silver_ensemble_amr_part4_empty_wiki.txt', 'r') as file:
# with open('/mnt/e/workspace/UCL_master_project/Get_wiki/AMR_without_wiki/ax_ensembled_AMR3_empty_wiki.amr', 'r') as file:
    dataset = file.read()
with open('dict_entities.json', 'r') as json_file:
    dict_entities = json.load(json_file)

dataset = dataset.split('\n\n')
print(len(dataset))
# Use ThreadPoolExecutor for parallel processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit tasks with indices
    futures = [executor.submit(get_wiki_with_index, i, amr, dict_entities, mediawikiapi, client) for i, amr in enumerate(dataset)]
    results = []
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        results.append(future.result())

# Sort results based on the original indices
results.sort(key=lambda x: x[0])
ordered_results = [result for _, result in results]

# Now ordered_results is in the same order as val_dataset
print(ordered_results)

825


 31%|███       | 254/825 [00:43<01:43,  5.52it/s]

Error wiki:  music Hyponotize


 44%|████▍     | 365/825 [00:59<03:30,  2.18it/s]

Error wiki:  product Prime


...

Error wiki:  wine Riesling Germany


 98%|█████████▊| 811/825 [02:14<00:01,  8.61it/s]

Error wiki:  wine Riesling Germany


100%|██████████| 825/825 [02:21<00:00,  5.83it/s]

['# ::snt The cat sat on the mat.\n# ::tok The cat sat on the mat .\n(s / sit-01\n    :ARG1 (c / cat)\n    :ARG2 (m / mat))', '# ::snt The cat did not sit on the mat.\n# ::tok The cat did not sit on the mat .\n(s / sit-01\n    :polarity -\n    :ARG1 (c / cat)\n    :ARG2 (m / mat))', "# ::snt When you've got no snow, it's really hard to learn a snow sport so we looked at all the different ways I could mimic being on snow without actually being on snow.\n# ::tok When you 've got no snow , it 's really hard to learn a snow sport so we looked at all the different ways I could mimic being on snow without actually being on snow .\n(h / hard-02\n    :ARG0-of (c / cause-01\n        :ARG1 (l2 / look-01\n            :ARG0 (w2 / we)\n            :ARG1 (w / way\n                :ARG1-of (d / differ-02)\n                :manner-of (p / possible-01\n                    :ARG1 (i2 / imitate-01\n                        :ARG0 (i / i)\n                        :ARG1 (b / be-located-at-91\n                

In [4]:
output_path = "/mnt/e/workspace/UCL_master_project/Get_wiki/amr_with_wiki"
output_name = "silver_ensemble_amr_part4_wiki.txt"
with open(output_path + '/' + output_name, 'w') as file:
    file.write('\n\n'.join(ordered_results))

In [ ]:

# /home/matthew/workspace/Applied_deep_learning/amr_workspace/data/ax_ensembled_AMR3_wiki.txt

with open('/mnt/e/workspace/UCL_master_project/Get_wiki/AMR_without_wiki/ax_ensembled_AMR3_wiki.txt', 'r') as f:
    dataset = f.read()

# split the data by '\n\n'
data = dataset.split('\n\n')
# Load the AMR string into a Penman graph
graph = penman.decode(data[0])

# decode the graph back to AMR string
amr_string = penman.encode(graph)
for i,amr in enumerate(tqdm(data)):
    if ':wiki -' in amr:
        graph = penman.decode(amr)
        wiki_names , wiki_idx = find_wiki_names(graph,amr)
        graph = add_wiki_tag(graph, wiki_idx, wiki_names)
        amr_string = penman.encode(graph)
        data[i] = amr_string

# write the data back to the file
with open('/mnt/e/workspace/UCL_master_project/Get_wiki/amr_with_wiki/ax_ensembled_AMR3_wiki_updated.txt', 'w') as f:
    f.write('\n\n'.join(data))

  0%|          | 34/72487 [00:19<8:27:32,  2.38it/s] 

camera Eyevision


  0%|          | 73/72487 [00:41<10:45:26,  1.87it/s]

championship nation Intercolleiate Sailing Association


  0%|          | 79/72487 [00:44<8:49:53,  2.28it/s] 

...

 69%|██████▉   | 50090/72487 [10:27:26<1:56:07,  3.21it/s] 

Oklo Fossil Reactors


 69%|██████▉   | 50220/72487 [10:28:45<3:12:06,  1.93it/s]

molecule Cu5Sn6
molecule Ag3Cu


 70%|██████▉   | 50398/72487 [10:30:42<4:53:54,  1.25it/s] 